### 라이브러리 로드

In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
from string import digits

### Data 읽기

![Integer Memory Layout](Fgr/TM_0_Complaint_Data.JPG)

In [2]:
import pandas as pd
senta = pd.read_csv('complaint.csv')
senta.head()

,대번호,소번호,문장,훼손/파손,분실,배송지연,오배송,"기타(임의반품, 불친절, 집하거부 등)",관련없음
0,1,1,2013. 11. 4. 신청인은 지인에게 선물하기 위해 햅쌀 20kg을 피신청인을...,NaN,NaN,NaN,NaN,NaN,y
1,1,2,"2013. 11. 8. 상하차 도중 포장이 훼손되어 배송불가하다는 연락을 받아, ...",y,NaN,NaN,NaN,NaN,NaN
2,1,3,신청인은 청원군 햅쌀 20kg 배송 이행과 함께 두 달 이상 지연된 것에 대한 배상...,NaN,NaN,NaN,NaN,NaN,y
3,1,4,\n* 운송장번호 : 8400 1322 3033,NaN,NaN,NaN,NaN,NaN,y
4,2,1,신청인은 2013.11. 9. 피신청인 동대문 도매업체를 통하여 의류를 주문하고 당...,NaN,NaN,NaN,NaN,NaN,y


### 민원 문장(senta_x)과 5개의 민원 유형(class) 분리

In [3]:
X_ = senta.iloc[:,2]
y_senta = senta.iloc[:,3:]

print(X_.head())
print(y_senta[:5])

0     2013. 11. 4. 신청인은 지인에게 선물하기 위해 햅쌀 20kg을 피신청인을...
1     2013. 11. 8. 상하차 도중 포장이 훼손되어 배송불가하다는 연락을 받아, ...
2    신청인은 청원군 햅쌀 20kg 배송 이행과 함께 두 달 이상 지연된 것에 대한 배상...
3                         \n* 운송장번호 : 8400  1322  3033
4    신청인은 2013.11. 9. 피신청인 동대문 도매업체를 통하여 의류를 주문하고 당...
Name: 문장, dtype: object
  훼손/파손   분실 배송지연  오배송 기타(임의반품, 불친절, 집하거부 등) 관련없음
0   NaN  NaN  NaN  NaN                   NaN    y
1     y  NaN  NaN  NaN                   NaN  NaN
2   NaN  NaN  NaN  NaN                   NaN    y
3   NaN  NaN  NaN  NaN                   NaN    y
4   NaN  NaN  NaN  NaN                   NaN    y


### 불필요한 기호와 숫자 제거

In [4]:
X = []
for i in range(len(X_)):
    tmp = X_[i]
    tmp = re.sub(r'[^\w]', ' ', tmp)
    tmp = re.sub("\d+", " ", tmp)
    tmp = re.sub(r'\s+',' ',tmp).strip()
    X.append(tmp)
    
X=pd.Series(X)
print(X.shape)
X[:3]

(2727,)


0         신청인은 지인에게 선물하기 위해 햅쌀 kg을 피신청인을 통해 선불로 택배 발송함
1    상하차 도중 포장이 훼손되어 배송불가하다는 연락을 받아 피신청인의 택배기사가 훼손된...
2    신청인은 청원군 햅쌀 kg 배송 이행과 함께 두 달 이상 지연된 것에 대한 배상을 요구함
dtype: object

### 민원 라벨 데이터 생성 (one hot -> categorical)

In [5]:
y = pd.Series(np.zeros(y_senta.shape[0]),dtype=int)
for i in range(y_senta.shape[1]):
    ind = y_senta.iloc[:,i] == 'y'
    y[ind] = i
    
print(y.head())
y.shape

0    5
1    0
2    5
3    5
4    5
dtype: int32


(2727,)

### Data를 훈련/테스트 데이터로 분리

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [7]:
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(2045,) (682,) (2045,) (682,)


In [8]:
X_train.head()

2104    다 신청인은 물품이 한국에 도착하지 아니하고 물품조회가 이루어지지 아니하여 발송인 ...
1657                                   군포BHub 간선하차 군포BHub
52                                          시계고장으로 A S 맡김
1323    다 신청인은 피신청인 과 피신청인 현대택배 안락대리점 를 통하여 확인한 결과 피신청...
2330                              이하넥스 서비스 신청번호 해당내용 신청번호
dtype: object

In [9]:
senta.iloc[[2104,1657],:]

,대번호,소번호,문장,훼손/파손,분실,배송지연,오배송,"기타(임의반품, 불친절, 집하거부 등)",관련없음
2104,432,4,"다. 신청인은 2015. 3. 13. 물품이 한국에 도착하지 아니하고, 물품조회가 ...",NaN,NaN,y,NaN,NaN,NaN
1657,341,4,20141204 21:02:06 군포BHub 간선하차 군포BHub(0318952584),NaN,NaN,NaN,NaN,NaN,y


In [10]:
y_train.head(2)

2104    2
1657    5
dtype: int32

### TF IDF

>간결하고 명확한 설명: https://nesoy.github.io/articles/2017-11/tf-idf

![Integer Memory Layout](Fgr/TM_1_TFIDF.JPG)

>자세하게 설명: http://euriion.com/?p=411929

![Integer Memory Layout](Fgr/TM_2_TFIDF.JPG)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
X_train_tfidf = vectorizer.fit_transform(X_train)
print(vectorizer.get_feature_names()[:100])

print("<<<shape of X_train_tfidf>>> : ",X_train_tfidf.shape)
print("<<<type of X_train_tfidf>>> : ",type(X_train_tfidf))
print("<<<X_train_tfidf[:10,:10]>>> :\n",X_train_tfidf[:10,:10])

['cj', 'cj대한통운', 'gs', 'kg', 'kgb', 'kgb택배', 'kg을', 'tv', '가격', '가격은', '가격을', '가능하다고', '가방', '가서', '가액', '가액을', '가지고', '같다고', '같은', '같이', '개가', '개는', '개당', '개를', '개월', '개의', '개중', '거부함', '거절함', '거주하는', '것으로', '것은', '것을', '것이', '것이라고', '것임', '결과', '결제함', '경기도', '경남', '경동택배', '경동택배를', '경비실에', '경우', '계속', '계약', '계약을', '고객센터', '고객센터로', '고객센터에', '고구마', '고시', '골프채', '공정거래위원회', '과실로', '과실을', '과실이', '과일', '과정에서', '관련', '관련법령', '구매', '구매자가', '구매자에게', '구매하고', '구매한', '구입', '구입가', '구입가격', '구입일', '구입하고', '구입하여', '구입한', '구입함', '국내', '규정에', '귤이', '그냥', '그대로', '그러나', '그런데', '그리고', '금액', '기다렸으나', '기다리라고', '기다리라는', '기사', '기사가', '기사에게', '기재', '기재되어', '기재된', '기재하여', '기재한', '기재함', '기준에', '기타', '김치', '까지', '깨져서']
<<<shape of X_train_tfidf>>> :  (2045, 814)
<<<type of X_train_tfidf>>> :  <class 'scipy.sparse.csr.csr_matrix'>
<<<X_train_tfidf[:10,:10]>>> :
 


In [12]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf',gamma=1).fit(X_train_tfidf, y_train)

In [13]:
print("훈련 세트 정확도: {:.2f}".format(svm.score(X_train_tfidf, y_train)))

훈련 세트 정확도: 0.90


In [14]:
X_test_tfidf = vectorizer.transform(X_test)
print("테스트 세트 정확도: {:.2f}".format(svm.score(X_test_tfidf, y_test)))

테스트 세트 정확도: 0.71


In [15]:
from sklearn.metrics import confusion_matrix

In [16]:
y_tr_pred = svm.predict(X_train_tfidf)
confusion_matrix(y_tr_pred, y_train)

array([[ 274,    2,    1,    0,    0,    3],
       [   0,  189,    0,    1,    0,    4],
       [   0,    0,   17,    0,    0,    0],
       [   0,    0,    0,    4,    0,    0],
       [   0,    0,    0,    0,   17,    0],
       [  43,   43,   28,   21,   54, 1344]], dtype=int64)

In [17]:
y_tt_pred = svm.predict(X_test_tfidf)
confusion_matrix(y_tt_pred, y_test)

array([[ 37,   3,   1,   1,   0,  12],
       [  0,  21,   0,   0,   0,   7],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [ 61,  54,  20,   9,  28, 428]], dtype=int64)

### Grid Search & Cross Validation

In [18]:
%%time
# wait 2~3 min
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], # 100
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]} # 0.01
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train_tfidf, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [19]:
grid_search.best_params_

{'C': 10, 'gamma': 0.1}

In [20]:
grid_search.best_score_

0.7550122249388753

In [21]:
print("훈련 세트 점수: {:.2f}".format(grid_search.score(X_train_tfidf, y_train)))

훈련 세트 점수: 0.93


In [22]:
print("테스트 세트 점수: {:.2f}".format(grid_search.score(X_test_tfidf, y_test)))

테스트 세트 점수: 0.72


In [23]:
print("최고 성능 모델:\n{}".format(grid_search.best_estimator_))

최고 성능 모델:
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


### Konlpy

https://wikidocs.net/21698

형태소 분석기: 
> Okt(Open Korea Text, 전 Twitter), 메캅(Mecab), 코모란(Komoran), 한나눔(Hannanum), 꼬꼬마(Kkma)

분해(tokenization) 방법:
> morphs(morpheme tokenization), pos(part of speech tagging), nouns

In [24]:
from konlpy.tag import Okt,Hannanum # Okt(Open Korea Text) <-- Twitter : Class Name Changed

In [25]:
X_train.iloc[6]

'변상확인요청서를 포함한 요청 서류 발송 후 주 내 보상 약속을 하였음'

### 한나눔(Hannanum)

In [26]:
# 형태소 토큰화(morphs)
hnm=Hannanum()
print(hnm.morphs(X_train.iloc[6]))

['변상확인요청서', '를', '포함', '하', 'ㄴ', '요청', '서류', '발송', '후', '주', '내', '보상', '약속', '을', '하', '었음']


In [27]:
# 품사부착(pos)
print(hnm.pos(X_train.iloc[6]))

[('변상확인요청서', 'N'), ('를', 'J'), ('포함', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('요청', 'N'), ('서류', 'N'), ('발송', 'N'), ('후', 'N'), ('주', 'N'), ('내', 'N'), ('보상', 'N'), ('약속', 'N'), ('을', 'J'), ('하', 'P'), ('었음', 'E')]


In [28]:
# 명사추출(nouns)
print(hnm.nouns(X_train.iloc[6]))

['변상확인요청서', '포함', '요청', '서류', '발송', '후', '주', '내', '보상', '약속']


### Okt: Open Korea Text

In [29]:
# 형태소 토큰화(morphs)
okt_tag = Okt()
print(okt_tag.morphs(X_train.iloc[6]))

['변상', '확인', '요청서', '를', '포함', '한', '요청', '서류', '발송', '후', '주', '내', '보상', '약속', '을', '하였음']


In [30]:
# 품사 부착(pos, Part of Speech)
print(okt_tag.pos(X_train.iloc[6]))

[('변상', 'Noun'), ('확인', 'Noun'), ('요청서', 'Noun'), ('를', 'Josa'), ('포함', 'Noun'), ('한', 'Josa'), ('요청', 'Noun'), ('서류', 'Noun'), ('발송', 'Noun'), ('후', 'Noun'), ('주', 'Noun'), ('내', 'Noun'), ('보상', 'Noun'), ('약속', 'Noun'), ('을', 'Josa'), ('하였음', 'Verb')]


In [31]:
# 명사추출(nouns)
print(okt_tag.nouns(X_train.iloc[6]))

['변상', '확인', '요청서', '포함', '요청', '서류', '발송', '후', '주', '내', '보상', '약속']


In [32]:
def okt_tokenizer(text):
    return okt_tag.morphs(text)

In [33]:
okt_vec = TfidfVectorizer(tokenizer=okt_tokenizer,min_df=5)
X_tr_okt = okt_vec.fit_transform(X_train)

In [34]:
print(okt_vec.get_feature_names()[:100])

['a', 'as', 'cj', 'dvd', 'gs', 'kg', 'kgb', 'lg', 'o', 's', 'tv', 'ㅇ', '가', '가격', '가능하다고', '가량', '가방', '가서', '가액', '가지', '가치', '각', '간', '간선', '감', '값', '같다고', '같은', '같이', '개', '개는', '개당', '개봉', '개요', '개월', '개인', '개중', '거래', '거부', '거절', '거제', '거주', '건', '건강', '것', '것임', '게', '게시판', '견적', '결과', '결정', '결제', '경', '경과', '경기도', '경남', '경북', '경비실', '경우', '계속', '계약', '계좌', '고', '고가', '고객', '고구마', '고시', '고장', '고지', '고함', '골프', '골프채', '곳', '공', '공연', '공정', '과', '과실', '과일', '과정', '관계', '관련', '관세', '광주', '교체', '교환', '구', '구매', '구멍', '구입', '구제', '구함', '국내', '권', '규정', '귤', '그', '그냥', '그대로', '그러나']


In [35]:
print("<<<shape of X_tr_okt>>> : ",X_tr_okt.shape)
print("<<<type of X_tr_okt>>> : ",type(X_tr_okt))
print("<<<X_tr_okt[:10,:10]>>> :\n",X_tr_okt[:10,:10])

<<<shape of X_tr_okt>>> :  (2045, 970)
<<<type of X_tr_okt>>> :  <class 'scipy.sparse.csr.csr_matrix'>
<<<X_tr_okt[:10,:10]>>> :
   (2, 0)	0.579361111175318
  (2, 9)	0.5875730645534857
  (8, 8)	0.3687037832553647


In [36]:
from sklearn.pipeline import make_pipeline
cplt_param_grid = {'svc__C': [1, 10, 100, 1000],
                   'svc__gamma': [0.01, 0.1, 1, 100]}
cplt_pipe = make_pipeline(TfidfVectorizer(tokenizer=okt_tokenizer,min_df=5), SVC())
cplt_grid = GridSearchCV(cplt_pipe, cplt_param_grid,cv=5)

In [40]:
# load the model from disk
cplt_grid = pickle.load(open(filename, 'rb'))
print("훈련 세트 점수: {:.2f}".format(cplt_grid.score(X_train, y_train)))

훈련 세트 점수: 0.94


In [42]:
print("테스트 세트 점수: {:.2f}".format(cplt_grid.score(X_test, y_test)))

테스트 세트 점수: 0.78


In [43]:
y_tr_pred = cplt_grid.predict(X_train)
confusion_matrix(y_tr_pred, y_train)

array([[ 289,    6,    0,    0,    1,   17],
       [   2,  201,    1,    0,    2,    9],
       [   0,    0,   39,    0,    0,    0],
       [   0,    0,    0,   22,    0,    0],
       [   0,    0,    0,    0,   46,    0],
       [  26,   27,    6,    4,   22, 1325]], dtype=int64)

In [44]:
y_tt_pred = cplt_grid.best_estimator_.predict(X_test)
confusion_matrix(y_tt_pred, y_test)

array([[ 77,   2,   2,   2,   1,  28],
       [  0,  48,   0,   1,   0,  15],
       [  1,   0,   3,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   1,   2,   0],
       [ 20,  28,  16,   6,  25, 404]], dtype=int64)

In [45]:
ind = y_tt_pred != y_test
print(ind[:10])

2711    False
332     False
195     False
2137    False
923      True
2707     True
776     False
1708    False
2013    False
2288    False
dtype: bool


In [47]:
print("X_test[4]: \n",X_test.iloc[4])
print("y_test[4]: ",y_test.iloc[4])
print("y_tt_pred[4]: ",y_tt_pred[4])
senta.columns[3:]

X_test[4]: 
 보낼때는 아무말없다가 파손되었다고하니 컴퓨터는 보내면 안되는 물품이라며 지사로 확인해보라고하며 책임을 회피함
y_test[4]:  5
y_tt_pred[4]:  0


Index(['훼손/파손', '분실', '배송지연', '오배송', '기타(임의반품, 불친절, 집하거부 등)', '관련없음'], dtype='object')

In [48]:
print("X_test[5]: \n",X_test.iloc[5])
print("y_test[5]: \n",y_test.iloc[5])
print("y_tt_pred[5]: \n",y_tt_pred[5])

X_test[5]: 
 KG 로지시의 택배약관에도 사고책임 및 보상 규정에서 손해액 배상을 한다고 하고 있으며 사업자 또는 사용인의 과실로 인하여 발생한 부분은 모든 규정에도 불구하고 손해를 배상함을 명시하고 있음
y_test[5]: 
 5
y_tt_pred[5]: 
 0


### Gensim Word2Vec

https://datascienceschool.net/view-notebook/6927b0906f884a67b0da9310d3a581ee/

https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/03/30/word2vec/

![Integer Memory Layout](Fgr/TM_3_W2V_ex.JPG)


> CBOW(Continuous Bag of Words) Embedding

![Integer Memory Layout](Fgr/TM_4_W2V_CBOW.JPG)


> Skip-Gram(SG) Embedding

![Integer Memory Layout](Fgr/TM_5_W2V_SG.JPG)



In [49]:
import gensim 
from gensim.models import Word2Vec

In [50]:
X_tr_mphs=[okt_tag.morphs(s) for s in X_train]
X_tt_mphs=[okt_tag.morphs(s) for s in X_test]
print("X_tr_mphs[:2]: \n",X_tr_mphs[:2])
print("X_tt_mphs[:2]: \n",X_tt_mphs[:2])

X_tr_mphs[:2]: 
 [['다', '신청', '인', '은', '물품', '이', '한국', '에', '도착', '하지', '아니하고', '물품', '조회', '가', '이루어지지', '아니하여', '발송', '인', '신청', '외', '김수지', '에게', '호주', '에서의', '물품', '발송', '여부', '확인', '을', '부탁', '함'], ['군포', 'BHub', '간선', '하차', '군포', 'BHub']]
X_tt_mphs[:2]: 
 [['신청', '인', '은', '경', '대', '한', '통운', '동', '광주', '지점', '은', '택배', '비', '금', '천원', '과', '가전제품', '을', '인수', '하면서', '파손', '면책', '에', '서명', '을', '하지', '않으면', '가져갈', '수', '없다며', '서명', '을', '요구', '하였으므로', '본인', '은', '물건', '을', '보내기', '위해', '서명', '을', '한', '사실', '이', '있음'], ['운송', '장', '번호']]


In [51]:
%%time
model = Word2Vec(X_tr_mphs)

Wall time: 146 ms


In [52]:
model.wv.similarity('신청', '물품')

0.8902616

In [53]:
model.wv.similarity('택배', '서명')

0.99543107

In [54]:
model.wv.most_similar("택배")

[('를', 0.9995439052581787),
 ('경', 0.9994050860404968),
 ('동', 0.9993618726730347),
 ('기사', 0.9993550777435303),
 ('부산', 0.9993494749069214),
 ('지점', 0.9993389248847961),
 ('대', 0.999316930770874),
 ('현대', 0.9993081092834473),
 ('대행', 0.9992659091949463),
 ('통운', 0.9992417097091675)]

In [55]:
model.wv.most_similar("파손")

[('훼손', 0.9997409582138062),
 ('제품', 0.9997328519821167),
 ('분실', 0.9996873140335083),
 ('에', 0.9996577501296997),
 ('하는', 0.9996424317359924),
 ('요청', 0.9996423721313477),
 ('사실', 0.9996275305747986),
 ('부분', 0.9996256232261658),
 ('과', 0.9995768070220947),
 ('사과', 0.9995695948600769)]

In [56]:
X_tr_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    X_tr_dict[key] = model.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
    # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)

In [57]:
%store X_tr_dict > X_tr_dict.txt

Writing 'X_tr_dict' (dict) to file 'X_tr_dict.txt'.


In [58]:
model.wv.vectors

array([[-0.49738663,  0.12606657,  0.18471774, ..., -0.42070967,
        -0.08481053,  0.65967464],
       [-0.5368251 ,  0.12702276,  0.22401915, ..., -0.3910539 ,
        -0.09769269,  0.6329188 ],
       [-0.38597324,  0.50184745,  0.28143334, ..., -0.684482  ,
         0.02626507,  0.806305  ],
       ...,
       [-0.06323787,  0.07567094,  0.047002  , ..., -0.11186374,
         0.00742382,  0.12635998],
       [-0.02991179,  0.02714918,  0.02282662, ..., -0.04296595,
        -0.00145514,  0.05530215],
       [-0.05174516,  0.07238609,  0.04541634, ..., -0.09597864,
         0.00387169,  0.11925457]], dtype=float32)

In [59]:
model.wv[['신청','제품']]

array([[-0.5368251 ,  0.12702276,  0.22401915,  0.32993618,  0.01066917,
         0.1478974 ,  0.31329438, -0.36615914, -0.24005356, -0.06175203,
        -0.32087246,  0.02673874,  0.15862945, -0.21094662, -0.10941812,
         0.5543546 , -0.03523646, -0.02206388,  0.40984178,  0.02939426,
        -0.30395958, -0.13485093, -0.3368669 ,  0.738099  , -0.3037211 ,
        -0.09991644, -0.51740015,  0.40043175,  0.01735786, -0.15377139,
        -0.08368897, -0.0307893 , -0.00201798,  0.24695738, -0.44683036,
        -0.15814564,  0.3380094 ,  0.22411872,  0.14994505,  0.4022854 ,
         0.7508758 , -0.11029902,  0.47598618,  0.04470073, -0.21169548,
        -0.17911357,  0.06031364, -0.02970409, -0.21693213, -0.5909086 ,
         0.19144952, -0.42639634,  0.01391671, -0.23499022,  0.01152144,
        -0.59654903,  0.05843896,  0.19332814, -0.29970217, -0.40113148,
         0.21778111,  0.15688719, -0.16409811,  0.17907536, -0.4500669 ,
        -0.14850417, -0.25789762, -0.5873179 ,  0.2

#### word averaging

출처: https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [60]:
import logging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors_norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [61]:
X_tr_wa = word_averaging_list(model.wv,X_tr_mphs)
X_tt_wa = word_averaging_list(model.wv,X_tt_mphs)

In [62]:
%%time
# wait 2 min
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [10, 100,1000], # 
              'gamma': [10, 100, 1000]} # 0.01
grid_search_wa = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)
grid_search_wa.fit(X_tr_wa, y_train)

Wall time: 1min 37s


In [63]:
grid_search_wa.best_params_

{'C': 100, 'gamma': 100}

In [64]:
grid_search_wa.best_score_

0.7154034229828851

In [65]:
print("훈련 세트 점수: {:.2f}".format(grid_search_wa.score(X_tr_wa, y_train)))

훈련 세트 점수: 0.77


In [66]:
print("테스트 세트 점수: {:.2f}".format(grid_search_wa.score(X_tt_wa, y_test)))

테스트 세트 점수: 0.69
